In [2]:
import pandas as pd
from DatabaseConnect import *
import warnings
warnings.simplefilter("ignore", UserWarning)

conn = connect()

{'host': 'localhost', 'database': 'phamkimhoang', 'user': 'postgres', 'password': 'postgres'}
Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 15.1 on aarch64-apple-darwin21.6.0, compiled by Apple clang version 14.0.0 (clang-1400.0.29.102), 64-bit',)


In [2]:
df = pd.read_csv('vnindex2.csv')
df

,Date,Price,Open,High,Low,Vol.,Change %
0,02/21/2023,"1,082.37","1,086.69","1,095.10","1,081.80",670.61K,-0.40%
1,02/20/2023,"1,086.69","1,059.74","1,088.01","1,059.56",682.29K,2.58%
2,02/17/2023,"1,059.31","1,058.29","1,062.25","1,053.26",465.08K,0.10%
3,02/16/2023,"1,058.29","1,048.20","1,058.29","1,047.45",475.80K,0.96%
4,02/15/2023,"1,048.20","1,038.64","1,051.89","1,034.39",585.76K,0.92%
...,...,...,...,...,...,...,...
471,04/01/2021,"1,216.10","1,194.59","1,217.26","1,192.83",672.84K,2.07%
472,03/31/2021,"1,191.44","1,186.36","1,196.45","1,186.36",703.08K,0.43%
473,03/30/2021,"1,186.36","1,179.64","1,187.29","1,172.85",780.73K,0.91%
474,03/29/2021,"1,175.68","1,169.65","1,175.77","1,165.78",660.95K,1.16%


In [3]:
df.dropna(inplace=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476 entries, 0 to 475
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      476 non-null    object
 1   Price     476 non-null    object
 2   Open      476 non-null    object
 3   High      476 non-null    object
 4   Low       476 non-null    object
 5   Vol.      476 non-null    object
 6   Change %  476 non-null    object
dtypes: object(7)
memory usage: 26.2+ KB


In [4]:
def fixVolumn(x):
    if 'K' in x:
        x = x.replace('K', '000')
        return float(x)
    elif 'M' in x:
        x = x.replace('M', '000000')
        return float(x)

df['Date'] = pd.to_datetime(df['Date'])
df['Price'] = pd.to_numeric(df['Price'].str.replace(',',''))
df['Open'] = pd.to_numeric(df['Open'].str.replace(',',''))
df['High'] = pd.to_numeric(df['High'].str.replace(',',''))
df['Low'] = pd.to_numeric(df['Low'].str.replace(',',''))
df['Change %'] = pd.to_numeric(df['Change %'].str.replace('%',''))    
df['Vol.'] = df['Vol.'].apply(fixVolumn)

In [5]:
df.info()
df = df[['Date', 'Open', 'Price', 'High', 'Low', 'Vol.', 'Change %']]
df.columns = ['tradingdate', 'open', 'close', 'high', 'low', 'volumn', 'pctchange']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476 entries, 0 to 475
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      476 non-null    datetime64[ns]
 1   Price     476 non-null    float64       
 2   Open      476 non-null    float64       
 3   High      476 non-null    float64       
 4   Low       476 non-null    float64       
 5   Vol.      476 non-null    float64       
 6   Change %  476 non-null    float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 26.2 KB


In [6]:
execute_values(conn, df, 'tbvnindex', type='INSERT')

The dataframe tbvnindex is inserted


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from datetime import timezone
from DatabaseConnect import *
import pandas as pd
import warnings
from IPython.display import clear_output
warnings.simplefilter("ignore", UserWarning)

conn = connect()

url_main = "https://www.tinnhanhchungkhoan.vn/sitemap.xml"

payload={}
headers = {}

response = requests.request("POST", url_main, headers=headers, data=payload)

soup = BeautifulSoup(response.text)
lst_urls_main = soup.find_all('loc')
print(len(lst_urls_main))


def crawlData(lst_urls):
    i = 1
    year = 2017
    for url in lst_urls:
        if i > 12:
            i = 1
            year += 1
            print("Year: ", year, " - Month: ", i)
            print("URL: ", url.text)
            print("///////////////////////////////////////////")
            print("///////////////////////////////////////////")
            print("///////////////////////////////////////////")
        else:
            print("Year: ", year, " - Month: ", i)
            print("URL: ", url.text)
            print(url)
            print("///////////////////////////////////////////")
            print("///////////////////////////////////////////")
            print("///////////////////////////////////////////")
        link = url.text
        if "news" in link:
            response2 = requests.request("POST", link, headers=headers, data=payload)
            soup2 = BeautifulSoup(response2.text)

            lst_item = soup2.find_all('loc')
            for item in lst_item[1::10]:
                try:
                    response3 = requests.request("GET", item.getText(), headers=headers, data=payload)
                    soup3 = BeautifulSoup(response3.text)
                except:
                    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                    print("'utf-8' codec can't decode byte 0xf4 in position 6: invalid continuation byte")
                    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

                try:
                    title = soup3.find_all('h1', {'class': 'article__header cms-title'})[0].getText()
                    category = soup3.find_all('li', {'class' : 'main-cate'})[0].getText()
                    date = soup3.find_all('time', {'class': 'time'})[0]['datetime']
                except:
                    bds = soup3.find_all('a', {'title': 'Bất động sản  - Đầu tư chứng khoán'})
                    if (len(bds) == 0):
                        if len(soup3.find_all('li', {'class' : 'main-cate'})) == 0:
                            print("Bài báo ko phân loại")
                        else:
                            print(item.text)
                            print('Other bug')
                    else:
                        bdsa = bds[0]['title']
                        print(bdsa, ' BĐS - no need')
                        print(item.text)

                datetime_object = datetime.strptime(date, '%Y-%m-%dT%H:%M:%S+0700')
                
                if "Chứng khoán" in category:
                    print(item.text)
                    d = pd.DataFrame({'tradingdate': [datetime_object], 'title': [title], 'category' : [category]})
                    execute_values(conn, d, 'tbnewssimple', type='INSERT')
        i = i+1

        clear_output(wait=True)

            
crawlData(lst_urls=lst_urls_main[75::-1])


Year:  2023  - Month:  4
URL:  https://www.tinnhanhchungkhoan.vn/sitemaps/categories.xml
<loc>https://www.tinnhanhchungkhoan.vn/sitemaps/categories.xml</loc>
///////////////////////////////////////////
///////////////////////////////////////////
///////////////////////////////////////////


In [3]:
sql = "select t.tradingdate::date as newsdate, t.title from tbnewssimple t order by t.tradingdate asc"
df_news = pd.read_sql_query(sql, conn)
df_news['title'] = df_news['title'].str.replace('[\s]+',' ',regex=True)
df_news_group = df_news.groupby("title").count().reset_index()
df_news = df_news.merge(df_news_group, on=['title'], how='left')
df_news = df_news[df_news['newsdate_y'] == 1]
df_news.columns = ['newsdate', 'title', 'count']
df_news

,newsdate,title,count
3,2009-06-01,Đảo nợ: Một vấn đề - nhiều cách hiểu!,1
4,2009-06-01,Giá gas tăng 5.000 đồng/bình 12kg,1
5,2009-06-01,"VAF đầu tư 9,4 triệu USD vào Masan",1
6,2009-06-01,FPT cổ phần hóa các công ty con,1
7,2009-06-01,PET nhập khẩu lô hàng phân kali đầu tiên,1
...,...,...,...
46320,2023-02-15,Cổ phiếu than vẫn còn dư địa tăng,1
46326,2023-02-19,Góc nhìn chuyên gia tuần tới: Cổ phiếu bất độ...,1
46330,2023-02-22,Chứng khoán châu Á đang bước vào nhịp điều ch...,1
46332,2023-02-23,Phục hồi thị trường trái phiếu: Ngân hàng sẽ ...,1


In [4]:
sql2 = "select * from ( select distinct on (t.title) t.newsdate::date as newsdate, t.title from tbnews t) t1 order by t1.newsdate asc"
df_news2 = pd.read_sql_query(sql2, conn)
df_news2

,newsdate,title
0,2016-08-05,UBS: 5 lý do để đầu tư vào TTCK mới nổi thời đ...
1,2016-08-05,"Khối ngoại tiếp tục “gom hàng” VNM, bán mạnh V..."
2,2016-08-05,Chứng khoán 6 tháng cuối năm: Bất động sản - V...
3,2016-08-06,"Loan tin có lãi rồi bất ngờ báo lỗ, những doan..."
4,2016-08-07,Thêm điều kiện với nhà đầu tư mua DN nhà nước
...,...,...
15304,2023-02-13,DSC: Sóng cổ phiếu midcap có thể xuất hiện khi...
15305,2023-02-13,Thị giá cổ phiếu VNZ vượt 1 triệu đồng sau 8 p...
15306,2023-02-13,"Cổ phiếu tâm điểm 14/2: EIB, KDC, PVD"
15307,2023-02-13,Thị trường chứng khoán (13/2): Kéo mạnh cuối p...


In [5]:
create_query(conn, "DELETE FROM tbnewsresult")
execute_values(conn, df_news[['newsdate', 'title']], 'tbnewsresult', type='INSERT')
execute_values(conn, df_news2, 'tbnewsresult', type='INSERT')

The query has been run
The dataframe tbnewsresult is inserted
The dataframe tbnewsresult is inserted
